ChEn-3170: Computational Methods in Chemical Engineering Fall 2018 UMass Lowell; Prof. V. F. de Almeida **03Oct2018**

# 07. Computational Stoichiometry
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \DeclareMathOperator{\rank}{rank}
$

---
## Table of Contents


---

## Introduction<a id="stoicmtrx"></a>
"Stoichiometry is essentially the bookkeeping of the material components of a chemical system." Rutherford Aris in Elementary Chemical Reactor Analysis. Computational stoichiometry is the matrix analysis of the stoichiometric matrix of a chemical reaction mechanism. We will use it two purposes: 
 + Evaluate sets of independent reactions
 + Evaluate reaction rates and species production rates

Recall [notes.](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/ErfDAD_jL1tHkDrx29w89-MBcOgK4JAnEYSheRoxkL_0sw?e=fpzG5K)

## Stoichiometric matrix<a id="stoicmtrx"></a>


In [ ]:
'''Open file, read data, and build reactions list'''

# open file in reading mode 'r' (default), text 't' (default)
finput = open('data/amonia-rxn.txt','rt')

reactions = list()
for line in finput:
    reactions.append( line.strip() )
for r in reactions: 
    i = reactions.index(r)
    print('r%s'%i,': ',r)

In [ ]:
'''Create species'''

species_tmp = list()  # temporary list for species
for r in reactions:
    left = r.split('<')
    right = r.split('>')
    species_left = left[0].split('+')
    species_right = right[1].split('+')
    species_rxn = species_left + species_right
    for i in species_rxn:
        species_tmp.append( i.split(' ')[1] )
print('species_tmp =',species_tmp)
species_filter = set(species_tmp) # filter species as a set
species = list( species_filter )  # convert species set to list 
print('species =\n',species)
print('# of species =',len(species))

In [ ]:
'''Create stoichiometric matrix'''

import numpy as np
s_mtrx = np.zeros((len(reactions),len(species)))
for r in reactions:
    left = r.split('<')
    left_terms = left[0].split('+')
    for t in left_terms:
        coeff = float(t.split(' ')[0])
        i_row = reactions.index(r)
        species_member = t.split(' ')[1]
        j_col = species.index(species_member)
        s_mtrx[i_row,j_col] = -1.0 * coeff
        
    right = r.split('>')
    right_terms = right[1].split('+')
    for t in right_terms:
        coeff = float(t.split(' ')[0])
        i_row = reactions.index(r)
        species_member = t.split(' ')[1]
        j_col = species.index(species_member)
        s_mtrx[i_row,j_col] = 1.0 * coeff
        
print('species',species)
print('s_mtrx =\n',s_mtrx)

In [ ]:
'''How many reactions are independent?'''

from in_house.matrix import lu_factorization
(p_mtrx, q_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx, 'complete' )

print('my rank =',rank)


print('numpy rank = ',np.linalg.matrix_rank( s_mtrx ))